In [ ]:
##Table of Contents
[Section 1: Introduction]
    -[Section 1.i: Historical Context](#section-1i-hisorical-context)
    -[Section 1.ii: Scope of Problem](#section-1ii-scope-of-problem)
[Section 2: Methodology](#section-2-methodology)
    -[Section 2.i: Processing](#section-2i-processing)
    -[Section 2.ii: Preprocessing](#section-2i-processing)


In [3]:
https://stackoverflow.com/questions/11948245/markdown-to-create-pages-and-table-of-contents/33433098#33433098


SyntaxError: invalid syntax (<ipython-input-3-1f5e0d95f412>, line 1)

In [ ]:
## Section 1: Introduction

In [ ]:
### Section 1.i: Historical Context

At the inception of the Ontario Birth Study in 2013, clinical data was stored on a Medidata Rave database (https://www.medidata.com/en/products/edc/) which was housed at the Applied Health Research Centre (AHRC; https://www.stmichaelshospital.com/research/ahrc/) at St. Michael’s Hospital. Medidata Rave was used as it was believed that the Ontario Birth Study data would be associated with the Ontario Health Study (https://www.ontariohealthstudy.ca/). Since the Ontario Birth Study is no longer associated with the Ontario Health Study and there were significant limitations with the Medidata Rave database, the Ontario Birth Study decided to move the clinical database from Medidata Rave to REDCap in late 2019. Both the Medidata Rave and REDCap database were developed by AHRC; however, the data transfer was done independently of ARHC. 



In [ ]:
### Section 1.ii: Scope of Problem

This notebook outlines the data migration process from Medidata Rave to REDCap completed in May 2020. The process outlined attempts to maximize data fidelity with minimal data cleaning. It was anticipated that the data cleaning proces would be done at a later date. Therefore, for the purposes of this project, it was assumed that all Medidata RAVE data received from AHRC was correct (i.e. no checking between label and coded data).

Although both the Medidata Rave and REDCap database were created by AHRC, there were differences in format and the numeric coding of variables. The RAVE data was given to the Ontario Birth Study in a wide format while the REDCap data needs to be in a long format to successfully import. Some columns, specifically date columns, forced the person entering the data to input a value even if one was not available. These instances had to be reinterpreted so the REDCap data reflected the intentions of the source data.

In [ ]:
## Section 2: Methodology
Broad overview of how the data was treated and checked

### Section 2.i: Processing
The RAVE database captures data in a wide format whereas the REDCap data is a long/narrow format. The `RedcapConv` class from `obs_clinic_migration` was created to aid in the conversion. The intializing of `RedcapConv` takes the following parameters: `ravestub_redcap_dict`, `stub_repeat`, `master_df`, `redcap_data_dict`, and, `recode_long`.

The `ravestub_redcap_dict` maps the name of the variable in REDCap to the associated variable in RAVE.

The `stub_repeat` parameter is the number of occurences in the RAVE dataset. For example if there is 'parameter_1' and 'parameter_2', the stub_repeat would be set to 2.

The `master_df` is the reference data frame that is being convert - in this case it is the RAVE dataframe which will be converted to the REDCap format.

redcap_data_dict

The `recode_long` Recode RAVE long dataframe based on REDcap data dictionar;            will change the output value based on the REDCAP DICITONARY


The function works by identifying the columns of interest in the RAVE clinic database. 

For example, suppose we are interested in the RAVE columns `rave_column_a` and `rave_column_b`. Two RAVE columns are availble in the RAVE data set: one with the data uncoded (i.e. represented as a string), another representing the data numerically coded. In the RAVE data set, the column representing the data numerically coded ended in 'STD' (e.g. `rave_parameter_a_STD`). 


Table 1. Example raw data from RAVE

In [ ]:
| Subject | rave_column_a | rave_column_a_STD | rave_column_b | rave_column_b_STD |
|---------|---------------|-------------------|---------------|-------------------|
| 100001  | Yes           | 1                 | Male          | 1                 |
| 100002  | No            | 0                 | Male          | 1                 |
| 100003  | Yes           | 1                 | Female        | 0                 |

In [ ]:
The uncoded column is referenced as the key in a dictionary (i.e. `ravestub_redcap_dict`) - the coded column is ignored. The corresponding dictionary value represents the name of the REDCap column. The dictionary is used as an argument for the `RedcapConv` class during initilization. With the dictionary
```python
{
        `rave_column_a`: `redcap_column_a`,
        `rave_column_b`: `redcap_column_b`
}
```

Table 1 will be converted into Table 2.
Table 2.

| obs_id | redcap_column_a | redcap_column_b |
|---------|-----------------|-----------------|
| 100001  | Yes             | Male            |
| 100002  | No              | Male            |
| 100003  | Yes             | Female          |

In [ ]:
If `recode_long` is set to `True`, then the initilization method will reference the column name in the REDCap data dictionary (i.e. `redcap_data_dict` parameter) and make the changes accordingly. For example, if the REDCap data dictionary listed the coding of `redcap_column_a` as '1, No | 2, Yes' and `redcap_column_b` as '1, Male | 2, Female | 3, Unknown', the table would result in Table 3.

Table 3.

| obs_id | redcap_column_a | redcap_column_b |
|---------|-----------------|-----------------|
| 100001  | 2               | 1               |
| 100002  | 1               | 1               |
| 100003  | 2               | 2               |

In [ ]:
The aforementioned example only works with columns were there is are no repeat instances (i.e. 'stub_repeat = 0'). Since the RAVE data is captured in a wide format, it needs to be converted to a long format before being imported into REDCap. For example, suppose the columns of interest are 'rave_column_a_1', 'rave_column_a_2', 'rave_column_b_1', and 'rave_column_b_2' where 'rave_column_a_1' and 'rave_column_a_2' are different iterations of the same parameter (similarly for 'rave_column_b_1' and 'rave_column_b_2'). 

RedcapConv has to manually enter number of instances > first surgery column is inconsistently named with the rest of the columns

Table 4.

| Subject | rave_column_a_1 | rave_column_a_1_STD | rave_column_a_2 | rave_column_a_2_STD | rave_column_b_1 | rave_column_b_1_STD | rave_column_b_2 | rave_column_b_2_STD |
|-|-|-|-|-|-|-|-|-|
| 100001 | Yes | 1 | No | 0 | Yes | 1 | No | 0 |
| 100002 | No | 0 | No | 0 | Yes | 1 | Yes | 1 |
| 100003 | Yes | 1 |  |  | Yes | 1 |  |  |


When 'stub_repeat' is set appropriately, Table 4 is converted to Table 5

Table 5.

| Subject | redcap_repeat_instance | rave_column_a | rave_column_b |
|-|-|-|-|
| 100001 | 1 | Yes | Yes |
| 100002 | 1 | No | Yes |
| 100003 | 1 | Yes | Yes |
| 100001 | 2 | No | No |
| 100002 | 2 | No | Yes |

In [ ]:
### Section 2.ii: Preprocessing
The RAVE columns are mapped one-to-one to REDCap columns.

rave_date_unknown
Add 'date unavailabe' column to RAVE dataframe.
    
    The new REDCap database has a column which indicates if date was available 
    for some variables. An equivalent column may not exist in the RAVE 
    database.


create_specify_col
Add 'please specify' column to RAVE dataframe
    
    The new REDCap database has a separate column for 'please specify'. In the
    RAVE database 'please specify' is intermingled in a column with the 
    associated labelled data. This function separates the 'please specify' 
    answer into a unique column based on the coded column.

In [ ]:
Section 2.ii: Postprocessing
There are instances when the processing is insufficient. Two additional methods were created to aid in migration.

#### `change_str` method
There were some instances where there were spelling variations between the RAVE database and the REDCap dictionary which arose as part of the database creation process. For example, if the REDCap data dictionary for column 'redcap_column_a' was '1, No | 2, Yes', the values in Table 6 would not be converted as part of the normal Processing process (Section 2.i) since the strings are different. If no changes were made, the associated values would remain as strings, not converted to integers, and there would be issues during the import process.

Table 6.

In [ ]:

| obs_id | redcap_column_a |
|---------|-----------------|
| 100001  | YES             |
| 100002  | NO              |
| 100003  | YES             |


In [ ]:
Once these discrepancies were identified, the 'change_str' method was used to alter the spelling of the values so they can be coded correctly. Passing the following argument to the 'change_str' can convert Table 6 to integers which can be imported without issues.
```
{'redcap_column_a': 
    {
        'YES' : 'Yes',
        'NO': 'No'   
    }
}

```

In [ ]:
#### `remove_na` method
In some instances, converting from the wide format in RAVE to the long format in REDCap resulted in additional rows with no data. The `remove_na` method removed these rows.


In [ ]:
### Section 2.iii: Data Verification
Three different processes were employed to ensure the fidelity of the data: code testing, double data entry, and the use of REDCap's internal data processing tools
To ensure the fidelity of the data, 3 different processes were employed


#### Code Quality Assurance
Code used to create the output files were functionally tested using `pytest` with the results available [here].

#### Data Verification
Double data entry process was used for [data verification](https://en.wikipedia.org/wiki/Data_verification) as suggested by technical articles[^1] and academic literature[^11][^12].

The data from 40 subjects was directly entered into both the RAVE and REDCap database. These 40 subjects were selected to maximize the number of columns covered in the RAVE data set. The first subject that was selected had the most amount of non-empty columns in the RAVE data set. The second subject selected had the most amount of non-empty columns after ignoring the non-empty columns the first subject had. The third subject selected had the most amount of non-empty columns after ignoring the non-empty columns the first and second subject had. This process continued until 40 subjects were selected and is outlined in `double_data_entry_subjects.py`. 

Once the RAVE data was converted into a format suitable to import into REDCap, it was then compared to the data directly entered into REDCap. If any discrepancies existed, these were rectified before the import process.

#### REDCap tools
The REDCap GUI provides the ability to visualize any discrepancies with the imported csv files (Figure 1). Any data that does not conform to the REDCap configuration will be flagged and the data will not be imported until the discrepancies have been resolved or the associated record is removed.


<img src="figures/data-imports-7.png">

In [ ]:
Figure 1. Example of REDCap data import quality check

https://docs.redcap.qmul.ac.uk/adding-data/import-data/

In [ ]:
### Section 2.iv: Prepare for Import and Save

For example,
The `prep_imp` method adds three different columns 'redcap_event_name', 'redcap_repeat_instrument', and the name of a completion column (i.e. `redcap_sheet_name_complete`). These values are not reviewed as part of the data verification process since they are reviewed during REDCap's 
import process. Finally, the data is saved locally and manually imported using REDCap's GUI.

In [ ]:
### Section 2.v: Overall check for missing columns

The data columns that were created as part of this process were compared to the data columns in REDCap (see SECTION X). This process was used to ensure all relevant columns were accounted - there were some columns in the REDCap project which were not in the RAVE database (e.g. Diabetes in Pregnancy) which were ignored.

In [ ]:
## Section 3: Example

In [ ]:
#### Preprocessing
```
rave_clinic = rave_date_unknown(rave_clinic, 'SURG_PROCS_AFTER_BASLIN_NY_', 'Yes', 'PROCEDURE_DT_', 2)
```

In [ ]:
#### Processing
```
ante_sur_dict = {
    'SURG_PROCS_AFTER_BASLIN_NY_': 'ante_sx_yn',
    'SURG_PROC_PERF_DURN_PREG_': 'ante_sx_class',
    'SPECIFY_PROCEDURE_': 'ante_sx_spec',
    'PROCEDURE_DT_DD_': 'ante_sx_day',
    'PROCEDURE_DT_MM_': 'ante_sx_month',
    'PROCEDURE_DT_YYYY_': 'ante_sx_year',
    'INDICATION_': 'ante_sx_indication',
    'COMMENT1_': 'ante_sx_comment',
    'PROCEDURE_DT_yn_date_': 'ante_sx_date_yn'     
}   
ante_sur = RedcapConv(ante_sur_dict, 2, master_df = rave_clinic)
```

In [ ]:
#### Postprocessing
```
ante_sur.change_str(
    {'ante_sx_class': {'Other, specify': 'Other'},}
)
```

In [ ]:
#### Data Verification
```
ante_sur.compare_conv_dde(redcap_clinic)
```

In [ ]:
#### Prepare for import and save
```
ante_sur.prep_imp('antenatal_arm_1', 'surgical_procedures_performed_during_pregnancy_complete', 'surgical_procedures_performed_during_pregnancy')    
ante_sur.data.to_csv('../data/processed/ante_sur.csv', index = False)
```

In [ ]:
[^1]: https://www.computerworld.com/article/2553608/avoiding-data-migration-delays.html
[^2]: https://academic.oup.com/jamia/article/9/6/600/1036696
[^11]: https://pubmed.ncbi.nlm.nih.gov/8235176/




https://pubmed.ncbi.nlm.nih.gov/23646084/

https://pubmed.ncbi.nlm.nih.gov/8235176/
https://influentialpoints.com/Training/Data_verification_Use_and_misuse.htm